In [ ]:
from gensim.corpora.wikicorpus import WikiCorpus
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from pprint import pprint
import multiprocessing
import gensim
print(gensim.__version__) # 0.12.4

### Read grouped data

In [ ]:
import pandas as pd
df1 = pd.read_pickle('grouped.pickle')

shuffle = True
if shuffle == True:
    df = df1.sample(frac=1)
else:
    df = df1

R = df['text']
G = df['gender']
S = df['specialty']
rating = df['scores']

print(R[0][1],rating[0],type(rating[0]))

In [ ]:
R2 = []
for i in range(len(R)):
    R2.append(R.iloc[i])
#print(R1[172])
G2 = []
for i in range(len(G)):
    G2.append(G.iloc[i])
print(G2[172])

S2 = []
for i in range(len(S)):
    S2.append(S.iloc[i])
print(S2[172])

rating2 = []
for i in range(len(rating)):
    rating2.append(rating.iloc[i])
print(rating2[172])

In [ ]:
import ast
x = rating2[13]
y = ast.literal_eval(x)
y[1]
len(R)

In [ ]:
scores,scores2 = [],[] # scores is -1,0,1. scores2 is raw average score.
for rr in rating2:
    r = ast.literal_eval(rr)
    score = 0
    counts = 0 # no. of valid ratings present out of 4
    for rr1 in r:
        try:
            r1 = int(ast.literal_eval(rr1))
        except:
            r1 = int(rr1)
        #print(r1)
        if(r1>0 and r1<=5):
            score += r1
            counts += 1
    try:
        score = score/counts
        #if score > 4.6:
        if score > 4:
            final_score = '1'
        #elif score < 3:
        elif score < 3.5:
            final_score = '-1'
        else:
            final_score = '0'
    except:
        final_score = '0'
    scores.append(final_score)
    scores2.append(score)
    #print(score)
len(scores)

In [ ]:
scores_freq = {}
for i in scores:
    try:
        scores_freq[i]+=1
    except:
        scores_freq[i]= 1
scores_freq

In [ ]:
R1,S1,G1,rating1 = [],[],[],[]
for i,s in enumerate(S2):
    if s == '-1':
        continue
    R1.append(R2[i])
    S1.append(S2[i])
    G1.append(G2[i])
    rating1.append(scores[i])

R1=R2
S1=S2
G1=G2
rating1=scores

### Picker

In [ ]:
# create reverse dictionary. Scores-> index. Specialties -> index.
revSpcl, revScore = {},{}
revSS = {}
for i in range(len(S1)):
    spcl = S1[i]
    score = rating1[i]
    try:
        revSpcl[spcl].append(i)
    except:
        revSpcl[spcl] = [i]
    try:
        revScore[score].append(i)
    except:
        revScore[score] = [i]
    # JUST TO CHECK IF SOME PAIR IS TOO RARE.
    try:
        revSS[spcl+':'+score+':'+G1[i]]+= 1
    except:
        revSS[spcl+':'+score+':'+G1[i]] = 1
#len(revScore['1'])

### Throw a dice for specialty. Get the corresponding reverse dict. Take intersection. Find a male one,  a female one.

In [ ]:
import random
def picker(roll_rating=False):
    spcl = '-1'
    while(spcl=='-1'): # Ignoring others.
        spcl = random.choice(list(revSpcl.keys()))
    spcl_cand = revSpcl[spcl]
    score = random.choice(list(revScore.keys()))
    score_cand = revScore[score]
    cand = set(spcl_cand).intersection(set(score_cand))
    #return list(cand)
    #return score_cand
    if roll_rating:
        return list(cand)
    else:
        return list(spcl_cand) # now only one coin toss, that decides specialty. work better? family male -1 has just 11 reviews.

In [ ]:
choice = random.choice(picker(roll_rating=True))
print(choice, rating1[choice], S1[choice])

### Tag and Learn Embeddings (Online)

In [ ]:
import gensim
docs = [] # each row is a tagged document.
for i in range(len(R1)):
    docs.append(gensim.models.doc2vec.TaggedDocument(R1[i], [S1[i],G1[i],rating1[i]]))

In [ ]:
docs_for_vocab = [] # contains no -1. Not updating docs because then reverese lists must also be updated.
for i in docs:
    if i[1][0] == '-1':
        continue
    else:
        docs_for_vocab.append(i)

In [ ]:
repeat = {}
def online(model,batch_size=1,repeat=repeat,roll_rating=False):
    indices = []
    batch = []
    #print('checkpoint3')
    for b in range(batch_size):
        pickm = False
        pickf = False
        cand = picker(roll_rating=roll_rating)
        while(not pickm or not pickf):
            choice = random.choice(cand)
            if G2[choice]=='m':
                if pickm:
                    continue
                else:
                    pickm=True
                    indices.append(choice)
            else:
                if pickf:
                    continue
                else:
                    pickf=True
                    indices.append(choice)
    for i in indices: # only 2: one m, one f.
        try:
            repeat[i]+=1
        except:
            repeat[i]=1
        #doc = gensim.models.doc2vec.TaggedDocument(R1[i], [S1[i],G1[i],rating1[i]])
        batch.append(docs[i])
    #print('checkpoint4')
    model.train(batch, total_examples=model.corpus_count)
    #print('checkpoint5')
        #return model # not needed. it changes anyway.

### Hyperparamter Search

In [ ]:
import gensim
import multiprocessing
from gensim.models.doc2vec import Doc2Vec

cores = multiprocessing.cpu_count()
size = 100

In [ ]:
def one_model(window,mc,sample,negative,batch_size,num_batches,roll_rating,index,iterations=1,hs=0,dm_concat=1,size=100):
    model = Doc2Vec(dm=0,size=size,window=window,min_count=mc,sample=sample,iter=iterations,workers=cores,hs=hs,negative=negative,dbow_words=1,dm_concat=dm_concat)
    #model = Doc2Vec(dm=1,size=size,window=window,min_count=mc,sample=sample,iter=iterations,workers=cores,hs=hs,negative=negative,dm_concat=dm_concat)
    model.build_vocab(docs_for_vocab)
    print(str(model))
    model.intersect_word2vec_format('w2v.bin')
    for i in range(1,num_batches):
        online(model,batch_size=batch_size,roll_rating=roll_rating) # each batch has twice as many as these reviews. m,f.
    version = str(i*batch_size*2)+'_'+str(model)
    version = version[:-1]+',dm_concat'+str(dm_concat)+',rr'+str(roll_rating)+')'
    #model.save('aug4/match_'+str(index)+'_'+version+'.model')
    print(i,end='')
    #print("model saved")
    #return str(model)
    return model

In [ ]:
index = 0
for hs in [0,1]: # 6912
    for dm_concat in [0,1]: # 3456
        for sample in [0,0.1,0.01,0.001]: # 1728
            for roll_rating in [True,False]: # 432 almost done.
                for window in [1,20,5,10,3,7]: # 216
                    for min_count in [0,50,20,30,10,5]: #36
                        for negative in [0,20,10,5,3,7]: # 6
                            for num_batches in [20]:
                                for batch_size in [10000]:
                                    #print(index,end='')
                                    m = one_model(window=window,mc=min_count,sample=sample,negative=negative,batch_size=batch_size,num_batches=num_batches,roll_rating=roll_rating,index=index,iterations=1,hs=hs,dm_concat=dm_concat,size=100)
                                    index+=1
                                    print(index)#,'\t',info)

## Save Model and Results

In [ ]:
m.save('aug7_roll_rating_match_UNDEBIASED_noRating.model')

### Words closest to 'm' and 'f' embeddings

In [ ]:
m.most_similar(positive=[m.docvecs['m']])
#m.most_similar(positive=[m.docvecs['f']])

### Words closest to specialty embeddings

In [ ]:
m.most_similar(positive=[m.docvecs['gyn']])
# and similarly for all individually.

### Specialties closest to 'm' and 'f' embeddings

In [ ]:
m.docvecs.most_similar(positive=[m.docvecs['m']])
#m.docvecs.most_similar(positive=[m.docvecs['f']])

### Positive adjectives for 'm' and 'f' embeddings

In [ ]:
m.docvecs.most_similar(positive=[m.docvecs['m'],m.docvecs['1']],negative=[m.docvecs['f']])
#m.docvecs.most_similar(positive=[m.docvecs['f'],m.docvecs['1']],negative=[m.docvecs['m']])

# t-SNE plot

In [ ]:
import gensim
m = gensim.models.doc2vec.Doc2Vec.load('aug7_match_july21.model')

In [ ]:
tokens = set()
for doc in m.docvecs.doctags:
    print(doc)
    for w in m.most_similar(positive=[m.docvecs[doc]],topn=5):
        #print(w[0])
        continue
# list of class tags / docvecs

### Words to be plotted

In [ ]:
#m.docvecs['1']
words = set()
for pair in m.most_similar(positive=[m.docvecs['m']],topn=5):
    words.add(pair[0])
for pair in m.most_similar(positive=[m.docvecs['f']],topn=5):
    words.add(pair[0])
    
for doc in m.docvecs.doctags:
    for pair in m.most_similar(positive=[m.docvecs[doc]],topn=5):
        words.add(pair[0])
    
len(words)
words = list(words)

In [ ]:
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
%matplotlib inline

def tsne_plot(model,words):
    "Creates and TSNE model and plots it"
    labels = []
    tokens = []

    for doc in model.docvecs.doctags:
        tokens.append(model.docvecs[doc])
        labels.append("#"+doc+"#")
    print(len(tokens),len(labels))
    
    ########## ADD NEW WORDS (specialty specific) ############
    gn = []
    gn.extend(['fertility','pregnancy','pregnant','delivery','egg','conceive','cycle','infertility','twin','baby'])
    gn.extend(['skin','acne','face','plastic','scar','dermatologist'])
    gn.extend(['disc','pt','fusion','knee','spine','shoulder','hip','lumbar','injury'])
    gn.extend(['endocrinologist','thyroid','diabetes','blood','lab'])
    gn.extend(['neurologist','spinal','brain','seizure','neurosurgeon','nerve'])
    gn.extend(['kid','pediatrician','child','pediatric'])
    gn.extend(['psychiatrist','mental','depression','therapist','illness'])
    gn.extend(['eye','cataract','retina','vision','lens','glass','laser'])
    gn.extend(['surgery','surgeon','cancer','tumor'])
    gn.extend(['septum','sinus','nose','ear','hearing','throat','infection','allergy','breath','breathing'])
    # also gender specific words
    gn.extend(['he','his','him','man'])
    gn.extend(['she','her','woman','herself'])
    # also rating specific words
    gn.extend(['great','good','excellent','caring','professional','wonderful','beautiful','caring'])
    gn.extend(['worst','rude','not','horrible','well','however','doctor'])
    
    for w in gn:
        try:
            tokens.append(model[w])
            labels.append(w)
        except:
            continue
    
    print(len(tokens),len(labels))
    print('now tsne-ing...')
    
    
    tsne_model = TSNE(perplexity=40, n_components=2, init='pca', n_iter=2500, random_state=23)
    new_values = tsne_model.fit_transform(tokens)

    return new_values, labels
    # RETURN CALLED.
    
    x = []
    y = []
    for value in new_values:
        x.append(value[0])
        y.append(value[1])
        
    plt.figure(figsize=(16, 16)) 
    for i in range(len(x)):
        plt.scatter(x[i],y[i])
        plt.annotate(labels[i],
                     xy=(x[i], y[i]),
                     xytext=(5, 2),
                     textcoords='offset points',
                     ha='right',
                     va='bottom')
    plt.show()

In [ ]:
%time new_values,labels = tsne_plot(m,words)

### Label and Colors

In [ ]:
size = {}
shape = {}
alpha = {}
color = {}

for i in m.docvecs.doctags:
    x = "#"+i+"#"
    alpha[x] = 1.0
    # remaining things below (in loop) are for specialties. specify differently for others.
    size[x] = 15
    shape[x] = 'x'
    color[x] = 'g'
    
# gender:
size["#f#"] = 20
shape["#f#"] = (3,2,0)
color["#f#"] = 'r'

size["#m#"] = size["#f#"]
shape["#m#"] = (3,2,180)
color["#m#"] = 'b'

# rating:
size["#1#"] = 15
shape["#1#"] = (3,1,0)
color["#1#"] = 'y'

size["#-1#"] = size["#1#"]
shape["#-1#"] = shape["#1#"]
color["#-1#"] = color["#1#"]

size["#0#"] = size["#-1#"]
shape["#0#"] = shape["#-1#"]
color["#0#"] = color["#-1#"]

# constant
size["#constant#"] = 15
shape["#constant#"] = '+'
color["#constant#"] = 'm'

### Show

In [ ]:
x = []
y = []
for value in new_values:
    x.append(value[0])
    y.append(value[1])

plt.figure(figsize=(16, 16)) 
for i in range(len(x)):
    try:
        sz = size[labels[i]]
        c = color[labels[i]]
        sh = shape[labels[i]]
        a = alpha[labels[i]]
        print(sz,c,sh,a)
    except:
        # DEFAULT. i.e. WORDS.
        sz = 10.0
        c = 'k'
        sh = 'o'
        a = 0.9
    plt.scatter(x[i],y[i],c=c,s=sz,marker=sh,alpha=a)
    plt.annotate(labels[i],
                 xy=(x[i], y[i]),
                 xytext=(5, 2),
                 textcoords='offset points',
                 ha='right',
                 va='bottom',color=c,fontsize=sz,alpha=a)
plt.show()